In [2]:
import dags

In [64]:
def utility(consumption, leisure, leisure_prefs):
    return consumption + leisure_prefs["weight"] * leisure ** leisure_prefs["exponent"]


def leisure_prefs(params):
    return params["preferences"]["leisure"]


def leisure(params):
    return params["time_budget"] - params["working_hours"]


def consumption(params):
    return params["hourly_wage"] * params["working_hours"]


def unrelated(working_hours):
    raise NotImplementedError()


In [65]:
from estimagic import maximize
from copy import deepcopy

In [66]:
start_params = {
    "preferences": {
        "leisure": {
            "weight": 0.95,
            "exponent": 2.0
        }
    },
    "hourly_wage": 25,
    "time_budget": 24 * 7 * 365,
    "working_hours": 2_000
}


In [67]:
model = dags.concatenate_functions(
    functions=[utility, unrelated, leisure, leisure_prefs, consumption],
    targets=["utility", "consumption"],
    return_type="dict"
)


In [68]:
model(params=start_params)

{'utility': 3342969280.0, 'consumption': 50000}

In [74]:
u = dags.concatenate_functions(
    functions=[utility, unrelated, leisure, leisure_prefs, consumption],
    targets="utility"
)

In [75]:
def return_all_but_working_hours(params):
    out = deepcopy(params)
    del out["working_hours"]
    return out

In [76]:
result = maximize(
    criterion=u,
    params=start_params,
    constraints=[
        {
            "selector": return_all_but_working_hours, 
            "type": "fixed"
        },
        {
            "selector": lambda p: [p["working_hours"], p["time_budget"]],
            "type": "increasing"
        }
    ],
    algorithm="scipy_lbfgsb"
)

/home/hmg/miniconda3/envs/software-pres/lib/python3.9/site-packages/pandas/core/frame.py:6248: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  diff = Index(subset).difference(self.columns)
/tmp/ipykernel_247800/1179560013.py:2: RuntimeWarning: overflow encountered in double_scalars
  return consumption + leisure_prefs["weight"] * leisure ** leisure_prefs["exponent"]
/home/hmg/miniconda3/envs/software-pres/lib/python3.9/site-packages/estimagic/differentiation/finite_differences.py:46: RuntimeWarning: invalid value encountered in subtract
  diffs = evals.pos - f0.reshape(1, dim_f, 1)


UserFunctionRuntimeError: An error occurred when evaluating criterion to calculate a numerical derivative during optimization.

In [52]:
result

Maximize with 1 free parameters terminated unsuccessfully after 21 criterion evaluations, 21 derivative evaluations and 0 iterations.

The value of criterion improved from 3342969280.0 to 1.2760887781340503e+23.

The scipy_lbfgsb algorithm reported: ABNORMAL_TERMINATION_IN_LNSRCH

Independent of the convergence criteria used by scipy_lbfgsb, the strength of convergence can be assessed by the following criteria:

                             one_step     five_steps 
relative_criterion_change     0.9375             1   
relative_params_change          0.75         0.999   
absolute_criterion_change  1.196e+23     1.276e+23   
absolute_params_change     2.749e+11     3.661e+11   

(***: change <= 1e-10, **: change <= 1e-8, *: change <= 1e-5. Change refers to a change between accepted steps. The first column only considers the last step. The second column considers the last five steps.)